### Import packages

In [ ]:
import glob
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import plotly.graph_objects as go
import matplotlib.pyplot as plt

### Read data from MSS simulation

In [ ]:
data_IMOB = []
mean_circuity_IMOB = {
        'circuity_driving-car': [],
        'circuity_driving-hgv': [],
        'circuity_foot-walking': [],
        'circuity_foot-hiking': [],
        'circuity_cycling-regular': [],
        'circuity_cycling-road': [],
        'circuity_cycling-mountain': [],
        'circuity_cycling-electric': [],
    }

for i, data_file in enumerate(sorted(glob.glob('Data/dist_time_lisbon_imob_*.csv'))):
    if 'circuity' in data_file:
        continue
    
    print('=====', data_file, '=====')
    df = pd.read_csv(data_file, index_col=0, skiprows=range(1,111515+1))

    df['circuity_driving-car'] = df['driving-car_dist'] / df['haversine_dist']/1000
    df['circuity_driving-hgv'] = df['driving-hgv_dist'] / df['haversine_dist']/1000
    df['circuity_foot-walking'] = df['foot-walking_dist'] / df['haversine_dist']/1000
    df['circuity_foot-hiking'] = df['foot-hiking_dist'] / df['haversine_dist']/1000
    df['circuity_cycling-regular'] = df['cycling-regular_dist'] / df['haversine_dist']/1000
    df['circuity_cycling-road'] = df['cycling-road_dist'] / df['haversine_dist']/1000
    df['circuity_cycling-mountain'] = df['cycling-mountain_dist'] / df['haversine_dist']/1000
    df['circuity_cycling-electric'] = df['cycling-electric_dist'] / df['haversine_dist']/1000

    mean_circuity_IMOB['circuity_driving-car'].append(df['circuity_driving-car'].mean(skipna=True))
    mean_circuity_IMOB['circuity_driving-hgv'].append(df['circuity_driving-hgv'].mean(skipna=True))
    mean_circuity_IMOB['circuity_foot-walking'].append(df['circuity_foot-walking'].mean(skipna=True))
    mean_circuity_IMOB['circuity_foot-hiking'].append(df['circuity_foot-hiking'].mean(skipna=True))
    mean_circuity_IMOB['circuity_cycling-regular'].append(df['circuity_cycling-regular'].mean(skipna=True))
    mean_circuity_IMOB['circuity_cycling-road'].append(df['circuity_cycling-road'].mean(skipna=True))
    mean_circuity_IMOB['circuity_cycling-mountain'].append(df['circuity_cycling-mountain'].mean(skipna=True))
    mean_circuity_IMOB['circuity_cycling-electric'].append(df['circuity_cycling-electric'].mean(skipna=True))
    
    if i == 0:
        drop_indices = np.random.choice(df.index, 11515, replace=False)
    
    df = df.drop(drop_indices)
    data_IMOB.append(df.reset_index(drop=True))
    
years = [str(i) for i in range(2013, 2021)]
data_aux = {}
for i, year in enumerate(years):
    data_aux[year] = data_IMOB[i]
data_IMOB = data_aux

df_IMOB_points = pd.read_csv('Data/df_IMOB_points.csv', index_col=0)

### Read parish data

In [ ]:
gdf_freguesias = gpd.read_file("Data/Lisboa_Freguesias/Lisboa_Freguesias_CAOP2015_TM06.shp")
gdf_freguesias = gdf_freguesias.to_crs(epsg=4326)
gdf_freguesias.geometry.index = gdf_freguesias['DICOFRE']
gdf_freguesias['Freguesia'].index = gdf_freguesias['DICOFRE'].astype('str')

In [ ]:
map = folium.Map([38.748662, -9.145801],
                 zoom_start=12,
                 tiles='cartodbpositron')
style_or = {'fillColor': '#F8C290', 'color': '#F8C290'}
unique_zones = gdf_freguesias['DICOFRE'].unique()

for i, zone in enumerate(unique_zones):
    mask = gdf_freguesias['DICOFRE'] == zone

    example_or = gdf_freguesias.loc[mask]
    folium.GeoJson(example_or,name='polygon_or',style_function=lambda x:style_or).add_to(map)

map

### Define Gini Coefficient computation function

In [ ]:
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq: http://www.statsdirect.com/help/content/image/stat0206_wmf.gif
    # from: http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    array = array.flatten() #all values are treated equally, arrays must be 1d
    if np.amin(array) < 0:
        array -= np.amin(array) #values cannot be negative
    array += 0.0000001 #values cannot be 0
    array = np.sort(array) #values must be sorted
    index = np.arange(1,array.shape[0]+1) #index per array element
    n = array.shape[0]#number of array elements
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array))) #Gini coefficient

## Compute Gini Coefficient for Driving, Cycling & Walking

In [ ]:
x = data_IMOB['2020']['circuity_driving-car'].to_numpy()
x = x[~np.isnan(x)]
print('Gini Coef. Driving: {:.4f}'.format(gini(x)))

x = data_IMOB['2020']['circuity_cycling-regular'].to_numpy()
x = x[~np.isnan(x)]
print('Gini Coef. Cycling: {:.4f}'.format(gini(x)))

x = data_IMOB['2020']['circuity_foot-walking'].to_numpy()
x = x[~np.isnan(x)]
print('Gini Coef. Walking: {:.4f}'.format(gini(x)))

#### Compute Gini Coefficient for each of Lisbon's parish

In [ ]:
data_IMOB_withFreguesias = data_IMOB['2020']
data_IMOB_withFreguesias['freguesia_or'] = df_IMOB_points['freguesia_or']
data_IMOB_withFreguesias['freguesia_de'] = df_IMOB_points['freguesia_de']

columns = ['walking', 'cycling', 'driving', 'freguesia']
gini_freguesias = pd.DataFrame(columns=columns)

for freguesia in df_IMOB_points['freguesia_or'].unique():
    data_freguesia = data_IMOB_withFreguesias[data_IMOB_withFreguesias['freguesia_or'] == freguesia]

    x_cycling = data_freguesia['circuity_cycling-regular'].to_numpy()
    x_cycling = x_cycling[~np.isnan(x_cycling)]
    x_walking = data_freguesia['circuity_foot-walking'].to_numpy()
    x_walking = x_walking[~np.isnan(x_walking)]
    x_driving = data_freguesia['circuity_driving-car'].to_numpy()
    x_driving = x_driving[~np.isnan(x_driving)]
    output = 'Gini Coef. for {} - W: {:.3f} / C: {:.3f} / D: {:.3f}'.format(
        str(gdf_freguesias['Freguesia'][str(freguesia)]).ljust(23),
        gini(x_walking),
        gini(x_cycling),
        gini(x_driving)
    )
    # print(output)
    data_row = {
        'walking': gini(x_walking),
        'cycling': gini(x_cycling),
        'driving': gini(x_driving),
        'freguesia': str(gdf_freguesias['Freguesia'][str(freguesia)])
    }
    gini_freguesias = gini_freguesias.append(data_row, ignore_index=True, sort=False)

### Plot Gini Coef for each parish

In [ ]:
gini_freguesias = gini_freguesias.sort_values(by=['cycling'], ascending=False)
    
fig = go.Figure()

fig.add_trace(go.Scatter(y=gini_freguesias['driving'],
                         x=gini_freguesias['freguesia'],
                    mode='lines+markers',
                    name='driving'))
fig.add_trace(go.Scatter(y=gini_freguesias['cycling'],
                         x=gini_freguesias['freguesia'],
                    mode='lines+markers',
                    name='cycling'))
fig.add_trace(go.Scatter(y=gini_freguesias['walking'],
                         x=gini_freguesias['freguesia'],
                    mode='lines+markers',
                    name='walking'))
        
fig.update_layout(
    title="Gini Coef for Feguesias",
    xaxis_title="Freguesias",
    yaxis_title="Gini Coef.",
    legend_title="Transport Mode",
    autosize=False,
    width=900,
    height=500,
    margin=dict(
        l=20,
        r=20,
        b=20,
        t=40,
        pad=2
    ),
    font=dict(
        family="Times New Roman",
        size=18,
        color="Black"
    ))
fig.update_xaxes(tickangle=-45)
fig.update_xaxes(
    ticktext=['Arroios', 'Av. Novas', 'Misericórdia', 'Sto. António', 'Areeiro', 'Sta. Maria Maior', 'Campo de Ourique', 'Alvalade', 'S. Vicente', 'Penha de França', 'Campolide', 'Estrela', 'S. Domingos de Benfica', 'Belém', 'Lumiar', 'Ajuda', 'Alcântara', 'Parque das Nações', 'Carnide', 'Olivais', 'Beato', 'Benfica', 'Marvila', 'Santa Clara'],
    tickvals=['Arroios', 'Avenidas Novas', 'Miseric', 'Santo Ant', 'Areeiro', 'Santa Maria Maior', 'Campo de Ourique', 'Alvalade', 'S1o Vicente', 'Penha de Fran', 'Campolide', 'Estrela', 'S1o Domingos de Benfica', 'Bel6m', 'Lumiar', 'Ajuda', 'Alc6ntara', 'Parque das Nas', 'Carnide', 'Olivais', 'Beato', 'Benfica', 'Marvila', 'Santa Clara']) 
fig.show()
plt.show()